In [1]:
import requests
import json
import pandas as pd
import datetime
from datetime import datetime
import calendar
import pickle
import joblib
import numpy as np

In [2]:
def get_forecast(air):
    loc = {'ATL': '33.6407,-84.4277', 'LAX': '33.9416,-118.4085', 'ORD': '41.9802,-87.90899', 'DFW': '32.8998,-97.0403', 'DEN': '39.8561,-104.6737', 'JFK': '40.6413,-73.7781', 'SFO': '37.6213,-122.3790', 'LAS': '36.0840,-115.1537', 'SEA': '47.4480,-122.3088', 'CLT': '35.2144,-80.9473'}
    url = 'https://api.weatherbit.io/v2.0/forecast/daily?'
    APIKey = '9688b0708e704469beb49279f95cfdc6'
    gets = []
    for key, value in loc.items():
        lat = '&lat=' + value.split(",")[0]
        long = '&lon=' + value.split(",")[1]
        key = '&key=' + APIKey
        n = url+lat+long+key
        gets.append(n)
    days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thurday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    k = ['ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'LAS', 'SEA', 'CLT']
    locs = dict(zip(k, gets))
    airport = air
    loc = locs[airport]
    grid = requests.get(loc)
    data = grid.json()
    df = pd.DataFrame(data['data'])
    df['Airport'] = airport
    df['precip'] = df['precip'] * 0.0393701
    df['wind_spd'] = df['wind_spd'] * 1.94384
    if 'vis' in df.columns:
        df['vis'] = df['vis'] * 0.621371
    df['temp'] = (df['temp'] *9/5) + 32
    df['dewpt'] = (df['dewpt'] *9/5) + 32
    df['pres'] = df['slp'] * 0.02953
    df["TempDate"] = pd.to_datetime(df["datetime"])
    df['DOW'] = df['TempDate'].dt.dayofweek
    df['day_of_week'] = df['DOW'].replace(days)
    df['temp_month'] = df['TempDate']
    df['month'] = df['TempDate'].dt.month.apply(lambda x: calendar.month_name[x])
    df2 = pd.get_dummies(data = df, prefix = 'day_of_week', columns = ['day_of_week'])
    df3 = pd.get_dummies(data = df2, prefix = 'month', columns = ['month'])
    df3['airtemp'] = df3['Airport']
    df4 = pd.get_dummies(data = df3, prefix = 'origin', columns = ['airtemp'])
    temp_list = ['vis', 'day_of_week_Monday', 'day_of_week_Tuesday', 'day_of_week_Wednesday', 'day_of_week_Thursday', 'day_of_week_Friday', 'day_of_week_Saturday', 'day_of_week_Sunday', \
                'month_April', 'month_August', 'month_December', 'month_February', 'month_January', 'month_July', 'month_June', 'month_March', 'month_May', 'month_November', 'month_October', 'month_September', \
                'origin_ATL', 'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_JFK', 'origin_LAS', 'origin_LAX', 'origin_ORD', 'origin_SEA', 'origin_SFO']
    for i in temp_list:
        if i not in df4.columns:
            df4[i] = 0
    df4['datetime'] = pd.to_datetime(df4["datetime"])
    df4['Current Time'] = datetime.now().strftime('%Y-%m-%d')
    df4 = df4.rename(columns = {'dewpt': 'dewp', 'rh': 'relh', 'wind_spd': 'wind_speed', 'pres': 'alti', 'slp': 'pressure', 'vis': 'visib'})
    df4 = df4[['Current Time', 'Airport', 'datetime', 'precip', 'temp', 'dewp', 'relh', 'wind_dir', 'wind_speed', 'pressure', 'visib',
             'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday',
             'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday',
             'day_of_week_Wednesday', 'month_April', 'month_August', 'month_December',
             'month_February', 'month_January', 'month_July', 'month_June', 'month_March',
             'month_May', 'month_November', 'month_October', 'month_September',
             'origin_ATL', 'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_JFK',
             'origin_LAS', 'origin_LAX', 'origin_ORD', 'origin_SEA', 'origin_SFO']]
    return df4

In [3]:
def compile_forecast():
    temp_list = []
    k = ['ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'LAS', 'SEA', 'CLT']
    for i in k:
        ports = get_forecast(i)
        temp_list.append(ports)
        df_future = pd.concat(temp_list)
    return df_future

In [4]:
def make_predictions():
    df_future = compile_forecast()
    gb_model = joblib.load('artifacts/gb_model.pkl')
    gb_model_cols = pickle.load(open('artifacts/gb_model.pkl', 'rb'))
    feats = df_future.iloc[:, 3:]
    gb_preds = gb_model.predict(feats)
    ls_model = joblib.load('artifacts/ls_model.pkl')
    ls_preds = ls_model.predict(feats)
    rf_model = joblib.load('artifacts/rf_model.pkl')
    rf_preds = rf_model.predict(feats)
    stacking_model = joblib.load('artifacts/stacking_model.pkl')
    stacking_preds = stacking_model.predict(feats)
    df_future['gb_model'] = gb_preds
    df_future['ls_model'] = ls_preds
    df_future['rf_model'] = rf_preds
    df_future['stacking_model'] = stacking_preds
    df_future = df_future.rename(columns = {'datetime': 'Forecast Date', 'precip': 'Precipitation (inches)', 'temp': 'Temperature (F)',
                            'dewp': 'Dew Point (F)', 'relh': 'Relative Humidity (%)',
                           'wind_dir': 'Wind Direction (Degrees)', 'wind_speed': 'Wind Speed (mph)',
                           'pressure': 'Pressure (mb)', 'visib': 'Visibility (miles)'})
    df_future['Current Time'] = pd.to_datetime(df_future['Current Time'])
    df_future['Forecast Date'] = pd.to_datetime(df_future['Forecast Date'])
    df_future['Precipitation (inches)'] = df_future['Precipitation (inches)'].astype('float')
    df_future['Temperature (F)'] = df_future['Temperature (F)'].astype('float')
    df_future['Dew Point (F)'] = df_future['Dew Point (F)'].astype('float')
    df_future['Relative Humidity (%)'] = df_future['Relative Humidity (%)'].astype('int')
    df_future['Wind Direction (Degrees)'] = df_future['Wind Direction (Degrees)'].astype('int')
    df_future['Wind Speed (mph)'] = df_future['Wind Speed (mph)'].astype('float')
    df_future['Pressure (mb)'] = df_future['Pressure (mb)'].astype('float')
    return df_future

In [5]:
today = datetime.today().strftime('%m-%d-%Y')
today = today.replace('-', "_")
fil_name = 'data/weather_' + today + '.csv'
make_predictions().to_csv(fil_name)